# Importing Libraries

In [ ]:
! pip install efficientnet-pytorch
! pip install --user torch==1.9.0 torchvision==0.10.0 torchaudio==0.9.0 torchtext==0.10.0

In [ ]:
%%time
import fastai
from fastai.vision.all import *
from pathlib import Path
from efficientnet_pytorch import EfficientNet

# Training on a subset of the training images.

In [ ]:
%%time
train = pd.read_csv("../input/trainsample/train-sample-split.csv", low_memory = False)

In [ ]:
train.head()

In [ ]:
path = Path("../input/happy-whale-and-dolphin/train_images/")

In [ ]:
whales = DataBlock(blocks = (ImageBlock, CategoryBlock),
                   get_x = ColReader("image", path),
                   get_y = ColReader("individual_id"),
                   splitter = ColSplitter("is_valid"),
                   item_tfms = Resize(440),
                   batch_tfms = [*aug_transforms(size = 224, min_scale = 0.75)])

In [ ]:
%%time
dls = whales.dataloaders(train)

In [ ]:
%%time

dls.show_batch()

In [ ]:
%%time
model = EfficientNet.from_name('efficientnet-b0')

In [ ]:
model._fc = nn.Linear(1280, dls.c)

# MAP@5 Metric

In [ ]:
def map_per_image(predictions, label):
    '''this function will calculate MAP@5 for a single image

    predictions = list of top 5 predictions for an image (Order does matter).
    label = true label
    '''
    try :
        return 1 / (predictions[:5].index(label) + 1)
    except ValueError :
        return 0.0



def map_per_set(predictions, labels):
    '''this function calculates MAP@5 for the whole set

    predictions = list of list of all the predictions for every image.
    labels = list of true labels 
    '''
    preds_sorted = []
    for i in predictions :
        temp = list(np.argsort(-i)[:5])
        preds_sorted.append(temp)
    return np.mean([map_per_image(p, l) for p,l in zip(preds_sorted, labels)])

In [ ]:
%%time
map5 = AccumMetric(map_per_set, to_np = False,  flatten = False)
learn = Learner(dls, model, metrics = [error_rate, map5])

# Finding Appropriate Learning Rate

In [ ]:
%%time
learn.lr_find()

In [ ]:
%%time
learn.fit_one_cycle(2, 1e-3)

In [ ]:
learn.recorder.plot_loss()

# Please Upvote if you liked my work 